In [13]:
pip install pandas pymysql faker openpyxl

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import random
from faker import Faker
import pymysql


fake = Faker()
Faker.seed(42)

# Generate 100 records
num_records = 100

data = {
    "job_title": [fake.job() for _ in range(num_records)],
    "date_of_application": [fake.date() for _ in range(num_records)],
    "name": [fake.name() for _ in range(num_records)],
    "email_id_1": [fake.email() for _ in range(num_records)],
    "email_id_2": [fake.email() for _ in range(num_records)],
    "phone_number_1": [fake.phone_number() for _ in range(num_records)],
    "phone_number_2": [fake.phone_number() for _ in range(num_records)],
    "current_location": [fake.city() for _ in range(num_records)],
    "preferred_location": [fake.city() for _ in range(num_records)],
    "total_experience": [random.randint(0, 20) for _ in range(num_records)],
    "curr_company_name": [fake.company() for _ in range(num_records)],
    "curr_company_designation": [fake.job() for _ in range(num_records)],
    "department": ["IT" if random.random() > 0.5 else "HR" for _ in range(num_records)],
    "role": ["Software Engineer" if random.random() > 0.5 else "Data Analyst" for _ in range(num_records)],
    "industry": ["Tech" if random.random() > 0.5 else "Finance" for _ in range(num_records)],
    "key_skills": ["Python, SQL" if random.random() > 0.5 else "Java, AWS" for _ in range(num_records)],
    "annual_salary": [random.randint(300000, 2000000) for _ in range(num_records)],
    "notice_period": [random.choice(["Immediate", "30 days", "60 days"]) for _ in range(num_records)],
    "resume_headline": ["Experienced in Data Science" for _ in range(num_records)],
    "summary": ["Data Scientist with 5 years of experience" for _ in range(num_records)],
    "ug_degree": ["B.Tech" for _ in range(num_records)],
    "ug_specialization": ["Computer Science" for _ in range(num_records)],
    "ug_university": [fake.company() for _ in range(num_records)],
    "ug_graduation_year": [random.randint(2010, 2023) for _ in range(num_records)],
    "pg_degree": ["M.Tech" for _ in range(num_records)],
    "pg_specialization": ["Data Science" for _ in range(num_records)],
    "pg_university": [fake.company() for _ in range(num_records)],
    "pg_graduation_year": [random.randint(2015, 2023) for _ in range(num_records)],
    "doctorate_degree": [None for _ in range(num_records)],
    "doctorate_specialization": [None for _ in range(num_records)],
    "doctorate_university": [None for _ in range(num_records)],
    "doctorate_graduation_year": [None for _ in range(num_records)],
    "gender": [random.choice(["Male", "Female"]) for _ in range(num_records)],
    "marital_status": [random.choice(["Single", "Married"]) for _ in range(num_records)],
    "home_town_city": [fake.city() for _ in range(num_records)],
    "pin_code": [fake.zipcode() for _ in range(num_records)],
    "date_of_birth": [fake.date_of_birth(minimum_age=22, maximum_age=50) for _ in range(num_records)],
    "permanent_address": [fake.address() for _ in range(num_records)],
}

df = pd.DataFrame(data)


In [12]:


df.fillna("", inplace=True)  # Handle null values

# Connect to MySQL
conn = pymysql.connect(host="localhost", user="root", password="cdac", database="job_portal1")
cursor = conn.cursor()

# Insert Data into Applicants Table
for _, row in df.iterrows():
    cursor.execute(
        "INSERT INTO applicants (name, email_id_1, phone_number_1, gender, marital_status, home_town_city, pin_code, date_of_birth, permanent_address) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (row["name"], row["email_id_1"], row["phone_number_1"], row["gender"], row["marital_status"], row["home_town_city"], row["pin_code"], row["date_of_birth"], row["permanent_address"]),
    )

conn.commit()
cursor.close()
conn.close()


In [16]:
import pymysql
import pandas as pd

# Connect to MySQL
conn = pymysql.connect(host="localhost", user="root", password="cdac", database="job_portal1")
cursor = conn.cursor()

# Read the Excel file
df = pd.read_excel("applicants_data.xlsx")
df.fillna("", inplace=True)  # Handle null values

# Insert Data into job_position (Static Data)
job_positions = [
    ("Software Engineer", "IT", "Mumbai"),
    ("Data Analyst", "IT", "Bangalore"),
    ("HR Manager", "HR", "Delhi"),
]

cursor.executemany(
    "INSERT IGNORE INTO job_position (position_name, department, location) VALUES (%s, %s, %s)",
    job_positions,
)
conn.commit()

# Retrieve job_position mapping
cursor.execute("SELECT position_id, position_name FROM job_position")
position_mapping = {name: pid for pid, name in cursor.fetchall()}

# Insert Data into Applicants and Related Tables
for _, row in df.iterrows():
    try:
        conn.begin()  # Start transaction
        
        # Insert into applicants table
        cursor.execute(
            """INSERT INTO applicants (name, email_id_1, phone_number_1, gender, marital_status, home_town_city, 
            pin_code, date_of_birth, permanent_address) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
            (row["name"], row["email_id_1"], row["phone_number_1"], row["gender"], row["marital_status"], 
             row["home_town_city"], row["pin_code"], row["date_of_birth"], row["permanent_address"]),
        )
        applicant_id = cursor.lastrowid  # Get the auto-generated applicant_id

        # Insert into experience table
        cursor.execute(
            """INSERT INTO experience (applicant_id, total_experience, curr_company_name, curr_company_designation, industry) 
            VALUES (%s, %s, %s, %s, %s)""",
            (applicant_id, row["total_experience"], row["curr_company_name"], row["curr_company_designation"], row["industry"]),
        )

        # Insert into education table
        cursor.execute(
            """INSERT INTO education (applicant_id, ug_degree, ug_specialization, ug_university, ug_graduation_year, 
            pg_degree, pg_specialization, pg_university, pg_graduation_year) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
            (applicant_id, row["ug_degree"], row["ug_specialization"], row["ug_university"], row["ug_graduation_year"], 
             row["pg_degree"], row["pg_specialization"], row["pg_university"], row["pg_graduation_year"]),
        )

        # Get position_id from mapping
        position_id = position_mapping.get(row["job_title"], None)

        # Insert into skills table
        cursor.execute(
            """INSERT INTO skills (applicant_id, position_id, key_skills, resume_headline, summary) 
            VALUES (%s, %s, %s, %s, %s)""",
            (applicant_id, position_id, row["key_skills"], row["resume_headline"], row["summary"]),
        )

        # Insert into applications table
        cursor.execute(
            """INSERT INTO applications (applicant_id, position_id, date_of_application, current_location, 
            preferred_location, annual_salary, notice_period) VALUES (%s, %s, %s, %s, %s, %s, %s)""",
            (applicant_id, position_id, row["date_of_application"], row["current_location"], 
             row["preferred_location"], row["annual_salary"], row["notice_period"]),
        )

        conn.commit()  # Commit transaction if everything is successful
        print(f"Inserted data for: {row['name']} (Applicant ID: {applicant_id})")

    except Exception as e:
        conn.rollback()  # Rollback in case of an error
        print(f"Error inserting data for {row['name']}: {e}")

cursor.close()
conn.close()


C:\Users\anike\AppData\Local\Temp\ipykernel_11388\1854601663.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)  # Handle null values


Inserted data for: Lisa Brandt (Applicant ID: 101)
Inserted data for: Nicole Vaughn (Applicant ID: 102)
Inserted data for: Jennifer Powers (Applicant ID: 103)
Inserted data for: Steven Hayes (Applicant ID: 104)
Inserted data for: Austin Smith (Applicant ID: 105)
Inserted data for: Leah Smith (Applicant ID: 106)
Inserted data for: Amy Jones (Applicant ID: 107)
Inserted data for: Gary Palmer (Applicant ID: 108)
Inserted data for: John Ryan (Applicant ID: 109)
Inserted data for: Sonya Johnston (Applicant ID: 110)
Inserted data for: John Russell (Applicant ID: 111)
Inserted data for: Matthew Gomez (Applicant ID: 112)
Inserted data for: Scott Brown (Applicant ID: 113)
Inserted data for: Maria Brown (Applicant ID: 114)
Inserted data for: Rebecca Rodriguez (Applicant ID: 115)
Inserted data for: Joshua Taylor (Applicant ID: 116)
Inserted data for: Joel Baxter (Applicant ID: 117)
Inserted data for: Robert Chase (Applicant ID: 118)
Inserted data for: Matthew Fernandez (Applicant ID: 119)
Inserte